<a href="https://colab.research.google.com/github/peterchang0414/randomized_priors/blob/main/randomized_priors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Randomized Prior Functions for Deep Reinforcement Learning
Based on Osband et al 2018: https://proceedings.neurips.cc/paper/2018/file/5a7b238ba0f6502e5d6be14424b20ded-Paper.pdf

Adapted to JAX from https://www.kaggle.com/code/gdmarmerola/introduction-to-randomized-prior-functions/notebook

Author: Peter G. Chang ([@peterchang0414](https://github.com/peterchang0414))